# ETL Pipeline for OMX stock prices
This is the ETL pipeline for the OMX25 stock prices to be used for the VA Exam in a Tableau dashboard.

The script is divided into three, subsequent sections:
1. Extract
2. Transform
3. Load

Each section will have undersections that will be seperated by the main dataframes that are being built:
* **Daily stock prices dataframe** 
* **Company key metrics dataframe** 

For each operation, a headline with the process will explain what will be performed. 

The script will produce 4 CSV files:

* **OMX_daily**, which is a long dataframe containing the close,volume,DMA30,50 and 200 for each of the OMX25 stocks from 2010-01-01 to today. 
* **OMX_info**, which is the entity of the characteristics of each of the companies.
* **OMX_financials**, which contains 14 financial metrics for each of the companies
* **Calender**, which is a date entity.

## Installing dependencies

In [70]:
#Installing dependencies
!pip install  yfinance
!pip install  pandas
!pip install  numpy


import pandas as pd
import yfinance as yf
import datetime
import numpy as np
 

# Extract
Two dataframes will be extracted from the yfinance API:
1. Daily stock prices
2. Company key metrics

## Daily stock prices dataframe
The Daily Stock Prices dataframe extract the data for the 25 danish stocks in the OMX25 index, by using the Yahoo API.
The purpose of this section is to extract the data and inspect it, and is the preliminary part of the ETL processes. 
The process is as follows: 

1. OMX25 Yahoo tickers are assigned statically assigned to a list.
2. The daily stock prices are retrieved from 2010-01-01 to today and assigned to the variable data_daily. 
3. Checking if the data has succesfully been retrieved
* *If yes, The data is assigned to a pandas dataframe called df_daily*
* *If no, an exception error will be raised.*
4. The dataframe is briefly inspected.

*The dataframe will be referred to as daily stock prices, although it will also include other metrics.*

In [71]:
#Creating a list for the Yahoo ticker names of the OMX stocks 
omx_tickers = ["COLO-B.CO", "CHR.CO", "TRYG.CO", "GN.CO", "ROCK-B.CO", "GMAB.CO", "FLS.CO", "NOVO-B.CO", "MAERSK-B.CO", "DSV.CO", "ISS.CO", "BAVA.CO", "NETC.CO", "MAERSK-A.CO", "CARL-B.CO", "ORSTED.CO", "RBREW.CO", "NZYM-B.CO", "DEMANT.CO", "NDA-DK.CO", "PNDORA.CO", "AMBU-B.CO", "DANSKE.CO", "VWS.CO", "JYSK.CO"]

today = datetime.date.today()
# Use the yfinance library to retrieve the data for each ticker.
data_daily = yf.download(omx_tickers, start='2010-01-01', end=today)

# Check if the data was retrieved successfully

if data_daily is not None:
    # Create a pandas dataframe from the retrieved data
    df_daily = pd.DataFrame(data_daily)
else:
    # Print an error message if the data was not retrieved successfully
    raise Exception("Error: failed to retrieve data")


[*********************100%***********************]  25 of 25 completed


### Data inspection 

Quick inspection of the dataframe. Further inspections will be provided in the transformation area.

In [72]:
print(f"Shape: {df_daily.shape}")
df_daily.head(10)

Shape: (3251, 150)


Adj Close                                                       \
           AMBU-B.CO     BAVA.CO   CARL-B.CO CHR.CO  COLO-B.CO  DANSKE.CO   
Date                                                                        
2010-01-04  5.007598   93.714554  313.378235    NaN  72.031273  75.971191   
2010-01-05  4.961657   93.714554  314.856659    NaN  69.485474  78.482643   
2010-01-06  5.191363  101.551308  309.391235    NaN  71.282509  79.424423   
2010-01-07  5.053540  101.551308  312.580841    NaN  72.181023  81.935875   
2010-01-08  5.099480   94.694145  313.378235    NaN  71.881531  83.191589   
2010-01-11  5.191363   97.632935  318.960083    NaN  73.029747  85.012398   
2010-01-12  5.168393   99.265587  314.973053    NaN  72.255905  84.447304   
2010-01-13  5.168393   94.612518  314.973053    NaN  71.404640  83.945030   
2010-01-14  5.283247  100.816612  311.783478    NaN  71.432281  86.016968   
2010-01-15  5.421070  107.408470  313.378235    NaN  72.330772  85.075172   

                                                         ...    Volume  \
            DEMANT.CO     DSV.CO      FLS.CO    GMAB.CO  ... NDA-DK.CO   
Date                                                     ...             
2010-01-04  78.000000  89.110435  293.015259  92.000000  ...    844025   
2010-01-05  77.000000  89.064034  289.625641  89.599998  ...   1142446   
2010-01-06  76.339996  93.287498  282.394440  88.750000  ...    163501   
2010-01-07  77.000000  93.751617  283.222992  88.000000  ...   1321570   
2010-01-08  78.019997  94.401360  283.147675  89.000000  ...    262138   
2010-01-11  78.239998  96.536308  282.469696  89.900002  ...   1179687   
2010-01-12  76.440002  94.679848  277.196991  89.599998  ...   1181918   
2010-01-13  77.099998  92.359276  278.703491  89.400002  ...    616080   
2010-01-14  77.279999  92.080788  285.482758  91.250000  ...   1173309   
2010-01-15  77.400002  90.502785  282.469696  97.500000  ...    792426   

                                                                      \
           NETC.CO  NOVO-B.CO NZYM-B.CO ORSTED.CO PNDORA.CO RBREW.CO   
Date                                                                   
2010-01-04     NaN  3683760.0    692320       NaN       NaN    63230   
2010-01-05     NaN  4167995.0    878675       NaN       NaN    23270   
2010-01-06     NaN  4686515.0    586310       NaN       NaN   173030   
2010-01-07     NaN   234445.0     70940       NaN       NaN    34395   
2010-01-08     NaN   294945.0   1063710       NaN       NaN    53665   
2010-01-11     NaN  5099605.0    883060       NaN       NaN   239895   
2010-01-12     NaN  3474315.0    526330       NaN       NaN   157150   
2010-01-13     NaN  3115135.0    500875       NaN       NaN   182930   
2010-01-14     NaN  3193525.0    447960       NaN       NaN   154315   
2010-01-15     NaN  5505225.0    651830       NaN       NaN   129590   

                                         
           ROCK-B.CO  TRYG.CO    VWS.CO  
Date                                     
2010-01-04     54658   264700   6862660  
2010-01-05      6387    52030   1607320  
2010-01-06      8474    27490    683490  
2010-01-07      3972  1566390   6365650  
2010-01-08     22034   101085   4956735  
2010-01-11      4413   646365   9473295  
2010-01-12     27598   475105   6171750  
2010-01-13     13165   593740  11293970  
2010-01-14     19676   774635   9146110  
2010-01-15     31920   520645   6106900  

[10 rows x 150 columns]

In [73]:
df_daily.dtypes

Adj Close  AMBU-B.CO    float64
           BAVA.CO      float64
           CARL-B.CO    float64
           CHR.CO       float64
           COLO-B.CO    float64
                         ...   
Volume     PNDORA.CO    float64
           RBREW.CO       int64
           ROCK-B.CO      int64
           TRYG.CO        int64
           VWS.CO         int64
Length: 150, dtype: object

## Company key metrics dataframe

The Company key metrics dataframe extract characteristics and chosen key financial metrics for the 25 danish stocks in the OMX25 index, by using the Yahoo API. 
The purpose of this section is to extract the data, and assign it to a Pandas dataframe.
1. A columns variable is assigned as a list, containing the header names of the chosen characteristics and metrics, as they are referred to by Yahoo.
2. By using a for-loop, a dataframe for the 25 companies will be created by extracting chosen dimensions and financial metrics from the yFinance API. 

- For each company the information will be retrieved from the Yahoo API, and it will be checked whether data has been retrieved.
    - *if yes, the data will be appended to the dataframe called info_df, as to end up with a wide format with one row for each company.*
    - *if not, an exception will be raised.*


In [74]:
# Creating an empty dataframe. 
info_df = pd.DataFrame()

# Assigning a list of company characteristics and finalcial metrics as named by yfinance.
columns = ['longName', 'symbol', 'industry','sector',  'ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow', 'revenueGrowth', 
           'operatingMargins', 'ebitda','debtToEquity', 'beta', 'returnOnEquity', 'totalCash','totalDebt', 'totalRevenue', 'totalCashPerShare']


# Loops through the comapnies
for ticker in omx_tickers:
    
    # Extracting the information for the given company using the Yahoo  api
    stock = yf.Ticker(ticker)
    info = stock.info
    
    # Check if the data was retrieved successfully
    if info is not None:

        # Create a new dictionary with the specified columns
        stock_new_dict = {key: info[key] for key in columns}

        # Creates a one-row dataframe based on the dictionary.
        df_dictionary = pd.DataFrame([stock_new_dict])
        
        # Appending the dataframe to the 
        info_df = pd.concat([info_df, df_dictionary], ignore_index=True)
    else:

        # Print an error message if the data was not retrieved successfully
        raise Exception(f"Error: failed to retrieve data for {ticker}")


### Data inspection
Quick inspection of the dataframe. Further inspections will be provided in the transformation area.

In [75]:
print(f"Shape: {info_df.shape}")
info_df.head(5)

Shape: (25, 18)


,longName,symbol,industry,sector,ebitdaMargins,profitMargins,grossMargins,operatingCashflow,revenueGrowth,operatingMargins,ebitda,debtToEquity,beta,returnOnEquity,totalCash,totalDebt,totalRevenue,totalCashPerShare
0,Coloplast A/S,COLO-B.CO,Medical Instruments & Supplies,Healthcare,0.33411,0.20842,0.68776,5098999808,0.188,0.30644,7544000000,225.615,0.239717,0.57181,633000000,18708000768,22578999296,2.982
1,Chr. Hansen Holding A/S,CHR.CO,Specialty Chemicals,Basic Materials,0.33268,0.18481,0.55542,308800000,0.118,0.26749,405200000,52.821,0.209557,0.13049,82800000,963400000,1218000000,0.629
2,Tryg A/S,TRYG.CO,Insurance—Diversified,Financial Services,0.12271,0.08608,0.17013,5902000128,0.168,0.10282,3615000064,15.165,0.341167,0.05498,1667000064,6682999808,29460000768,2.602
3,GN Store Nord A/S,GN.CO,Medical Devices,Healthcare,0.11320,0.03825,0.50414,318000000,0.241,0.07775,1983000064,223.702,1.026635,0.11108,870000000,16110999552,17517000704,6.799
4,Rockwool A/S,ROCK-B.CO,Building Products & Equipment,Industrials,0.16157,0.06378,0.57433,337000000,0.267,0.10438,613000000,None,1.065804,0.09871,210000000,0,3793999872,9.800


# Transform

## Daily stock prices dataframe

The goal of the transformation of the daily stock prices dataframe, is to end up with a long tidy table format.
Each row should represent a single observation, which for this dataframe is the close price and volume, for every stock for every date. Addtionally variables of interest are DMA30, DMA50 and DMA200 which also will be calculated.

The process is as follows:

1. Converting from wide to long format
2. The data will be inspected
3. A quality check will be performed, to ensure that there are no discrepancy between the tickers retrieved from yfinance and the omx 25 tickers.
4. Data cleaning
* **Exploring missing values in the dataframe.** Here it is found that a large amount of missing values are due to companies not being publicly traded.
* **Exploring missing values between non-missing values**, meaning that the value could be considered sentinal. In this case only value is found, which could be due to the stock not being traded that day or Yahoo do not have the observation.
* **Removing missing values**. However, if the data contains more than 10 missing values, further actions will be requiered by the user.
5. 3 Technical metrics are added to the dataframe: DMA30, DMA50 and DMA200 calculating the rolling average.  



### Converting from wide to long format

150 coumns is messy, and for our analysis we will only need volume and close prices. 

We will tidy the dataframe by converting to long format, with a column for close and volume as they are the only ones needed for our analysis.
* The 'volume' and 'adj. close' columns will be melted as seperate dataframes, have the columns renamed, and joined as  **df_daily_combined**. 


In [76]:
# Creating two seperate dataframes for sales volume and close price.
df_daily_volume = df_daily['Volume'].reset_index().melt(id_vars=["Date"])
df_daily_close = df_daily['Adj Close'].reset_index().melt(id_vars=["Date"])


# Renaming columns
df_daily_volume = df_daily_volume.rename(columns={"variable": "Ticker", "value": "Volume"})
df_daily_close = df_daily_close.rename(columns={"variable": "Ticker", "value": "Close"}) 

# Joining the two tables
df_daily_combined = pd.merge(df_daily_close, df_daily_volume, on=["Date", "Ticker"])

### Data inspection of the long format.

In [77]:
df_daily_combined.head(10)

,Date,Ticker,Close,Volume
0,2010-01-04,AMBU-B.CO,5.007598,124420.0
1,2010-01-05,AMBU-B.CO,4.961657,910560.0
2,2010-01-06,AMBU-B.CO,5.191363,51220.0
3,2010-01-07,AMBU-B.CO,5.053540,39060.0
4,2010-01-08,AMBU-B.CO,5.099480,69160.0
5,2010-01-11,AMBU-B.CO,5.191363,54420.0
6,2010-01-12,AMBU-B.CO,5.168393,131960.0
7,2010-01-13,AMBU-B.CO,5.168393,64120.0
8,2010-01-14,AMBU-B.CO,5.283247,459220.0
9,2010-01-15,AMBU-B.CO,5.421070,142440.0


In [78]:
print(f"Shape: {df_daily_combined.shape}")

df_daily_combined.dtypes

Shape: (81275, 4)


Date      datetime64[ns]
Ticker            object
Close            float64
Volume           float64
dtype: object

### Quality check: Checking for Ticker Symbol Consistency

The following code checks if the unique ticker symbols in the df_daily_combined DataFrame are the same as the 25 ticker symbols in the omx_tickers list. If they are not equal, it raises an exception with an error message.

In [79]:
#First data quality check 

# Extract an array of unique ticker symbols in df_daily_combined
df_tickers = df_daily_combined["Ticker"].unique()

# Checking whether all the tickers of the extract are equal to the omx25 tickers.
if np.array_equal(np.sort(df_tickers),np.sort(omx_tickers)) == False:
    raise Exception(f"Error with Dataextract from yFinance, as companies extracted are not equal to companies provided")





## Data Cleaning 
1.  Checking for missing values and exploratory investigation 
2.  Checking for missing values between two non-missing values.
3.  Removing missing values if missing values between non-missing values < threshhold. 

In [80]:
#Checking for missing values and exploratory investigation. 

# Counting the number of missing values in each column for the long dataframe
null_counts_long = df_daily_combined.isna().sum()
print(null_counts_long)
#This gives 4942 for all continious columns


# Counting the number of missing values in each column for the wide/imported dataframe
null_counts_wide = df_daily[['Adj Close', 'Volume']].isna().sum()
print(sum(null_counts_wide))

#Gets 9884 /2 = 4942, so long == Wide in terms of NA. I see the largest amount of NaN values for Net Company
#Which makes logical sense, as they have not been publicly traded for the whole duration. Furhtermore, volume = adj Close, 
#So it can be assumed that NAN is across all continuous columns (meaning either all are NaN or None are NaN). 
#Deleting missing rows is suitable for the long DF, as records will not be lost for the other stocks, but first I will check if there 
#are any values that are not missing due to the stock not being traded





Date         0
Ticker       0
Close     4942
Volume    4942
dtype: int64
9884


### Checking for Missing values that are between two Non-NaN values

In [81]:
# Creating shift_up and shift_down columns, that stores the data of the previous and next close price.

df_daily_combined["Close_shift_up"] = df_daily_combined["Close"].shift(1)
df_daily_combined["Close_shift_down"] = df_daily_combined["Close"].shift(-1)

# Creating a boolean mask indicating rows with missing values between non-missing values
mask = df_daily_combined["Close"].isna() & df_daily_combined["Close_shift_up"].notna() & df_daily_combined["Close_shift_down"].notna()

# Using the mask to filter the rows with missing values between non-missing values
df_filtered = df_daily_combined[mask]

# The resulting DataFrame will only include rows with missing values between non-missing values
df_filtered

# Here I find only one missing observation, for Novo Nordisk, which proves a pretty sound dataframe. 
# It is visible from the shift_up and shift_down that the previous and next close prices are non NaN



,Date,Ticker,Close,Volume,Close_shift_up,Close_shift_down
55355,2010-05-14,NOVO-B.CO,NaN,NaN,72.416992,72.8927


### Removing missing values.
If the missing values between two non-missing values are below a treshhold of 10, all of NaN values will be removed from the data frame to tidy the data. 

If is above 10, specific action will be requiered and the user will be informed by raising an exception.

In [82]:
# dropping all missing values if the amount of missing values between non-missing values are below 10
missing_values = df_filtered.shape[0]

missing_values_threshold = 10
if missing_values < missing_values_threshold:
    df_daily_combined = df_daily_combined.dropna().reset_index(drop=True)
else:
    raise Exception(f"There was found {missing_values} missing valuesbetween non-missing values. Additionally cleaning actions are required in pipeline.")


# Selecting only the neccessary columns
df_daily_combined = df_daily_combined[['Date','Ticker','Close','Volume']]


### Checking and removing  duplicates
Having duplicates will constitute untidy data, and could harm our analysis, for example with aggregations in a line chart in Tableau.

0 duplicates are found.

In [83]:
# Finding total duplicates
duplicates = df_daily_combined.duplicated()
num_duplicates = duplicates.sum()

### Adding DMA Technical metrics 

The technical metrics needed for our analysis are DMA30, 50 and 200.

1. The function calc_rolling_mean will be defined, which adds the 3 columns by using the rolling and mean methods.
2. By grouping the dataframe by tickers, the functions is then applied and iterates over the different tickers in the dataframe.
3. The final dataframe will be investigated, to ensure that the DMA columns has been added asuccesfully.

In [84]:
# Assigning a function that calculate the three DMA metrics.
def calc_rolling_mean(df):
    df["DMA30"] = df["Close"].rolling(30).mean()
    df["DMA50"] = df["Close"].rolling(50).mean()
    df["DMA200"] = df["Close"].rolling(200).mean()
    return df

# Applying the function to the dataframe, and assigning it to a new name: df_daily_final
df_daily_final = df_daily_combined.groupby("Ticker", group_keys=False).apply(calc_rolling_mean)

In [85]:
#Checking whether the DMA metrics has been succesfully applied. For the DMA30 it should have 3 NaN value for the DMA30, and 3 non-NaN
df_daily_final_filtered = df_daily_final[df_daily_final['Ticker'] == 'NOVO-B.CO']
df_daily_final_filtered.iloc[26:32]

,Date,Ticker,Close,Volume,DMA30,DMA50,DMA200
52032,2010-02-09,NOVO-B.CO,56.622036,7456185.0,NaN,NaN,NaN
52033,2010-02-10,NOVO-B.CO,56.576756,4235715.0,NaN,NaN,NaN
52034,2010-02-11,NOVO-B.CO,57.285873,5660725.0,NaN,NaN,NaN
52035,2010-02-12,NOVO-B.CO,58.372131,8456570.0,53.971608,NaN,NaN
52036,2010-02-15,NOVO-B.CO,59.051060,5306880.0,54.240159,NaN,NaN
52037,2010-02-16,NOVO-B.CO,59.247181,5071570.0,54.535978,NaN,NaN


## Company key metrics Dataframe

The goal of this transformation is to clean the compny key metrics dataframe, referred to as info_df, so the final dataframe will be in wide format, with a row for each company. After the transformation, the dataframe will be split into two tables, one with financial metrics and another for the characteristics.

The process for this transformation is as follows:

1. Two quality checks will be performed
2. The data will be inspected, and missing values will be checked for.
3. Data cleaning by removing NaN values.



### Quality check

1. Checking whether all 25 companies have been extracted from the dataframe. Raises an error if not.

2. Checking whether all metrics have been retrived. Raises and error if not.

In [86]:
# Check 1: tickers

# Extract the unique ticker values as a numpy array.
df_symbols = info_df["symbol"].unique()

# Checking whether all the symbols of the extract are equal to the symbols provided.
if np.array_equal(np.sort(df_symbols),np.sort(omx_tickers)) == False:
    raise Exception(f"Error with Dataextract from yFinance, as companies extracted are not equal to companies provided")


# Check 2: headers.

df_columns = info_df.columns 

if np.array_equal(np.sort(df_columns),np.sort(columns)) == False:
    raise Exception(f"Not all columns has been succesfully retrieved in the info_df dataframe.")



### Data inspection
Below the shape, sum of NaN values and datatypes are printed

It is seen that there in total are 4 missing values; some of the columns of numerical data contains "None", which is a data error retrived from yfinance and results in the datatypes being objects (integers). 

This will be cleaned. 

In [87]:
print(f"shape: {info_df.shape}")
print(f"na values: {info_df.isna().sum()}")
info_df.head(5)

shape: (25, 18)
na values: longName             0
symbol               0
industry             0
sector               0
ebitdaMargins        0
profitMargins        0
grossMargins         0
operatingCashflow    0
revenueGrowth        0
operatingMargins     0
ebitda               3
debtToEquity         4
beta                 0
returnOnEquity       0
totalCash            0
totalDebt            0
totalRevenue         0
totalCashPerShare    0
dtype: int64


,longName,symbol,industry,sector,ebitdaMargins,profitMargins,grossMargins,operatingCashflow,revenueGrowth,operatingMargins,ebitda,debtToEquity,beta,returnOnEquity,totalCash,totalDebt,totalRevenue,totalCashPerShare
0,Coloplast A/S,COLO-B.CO,Medical Instruments & Supplies,Healthcare,0.33411,0.20842,0.68776,5098999808,0.188,0.30644,7544000000,225.615,0.239717,0.57181,633000000,18708000768,22578999296,2.982
1,Chr. Hansen Holding A/S,CHR.CO,Specialty Chemicals,Basic Materials,0.33268,0.18481,0.55542,308800000,0.118,0.26749,405200000,52.821,0.209557,0.13049,82800000,963400000,1218000000,0.629
2,Tryg A/S,TRYG.CO,Insurance—Diversified,Financial Services,0.12271,0.08608,0.17013,5902000128,0.168,0.10282,3615000064,15.165,0.341167,0.05498,1667000064,6682999808,29460000768,2.602
3,GN Store Nord A/S,GN.CO,Medical Devices,Healthcare,0.11320,0.03825,0.50414,318000000,0.241,0.07775,1983000064,223.702,1.026635,0.11108,870000000,16110999552,17517000704,6.799
4,Rockwool A/S,ROCK-B.CO,Building Products & Equipment,Industrials,0.16157,0.06378,0.57433,337000000,0.267,0.10438,613000000,None,1.065804,0.09871,210000000,0,3793999872,9.800


In [88]:
info_df.dtypes


longName              object
symbol                object
industry              object
sector                object
ebitdaMargins        float64
profitMargins        float64
grossMargins         float64
operatingCashflow      int64
revenueGrowth        float64
operatingMargins     float64
ebitda                object
debtToEquity          object
beta                 float64
returnOnEquity       float64
totalCash              int64
totalDebt              int64
totalRevenue           int64
totalCashPerShare    float64
dtype: object

### Data cleaning
The purpose of the data cleaning is to convert the None values into NaN and cast the column types to float. 
This will be done by:
1. Assigning an id_column list, which are the characteristics of the companys, and an mettric_columns list which are the numerical metrics.
2. Convert the dataframe to long format, by melting the financial metrics columns into a metric and value column. The purpose of converting to long is that:
* Missing values can be removed for just a given metric, without deleting the others.
* Having all of the metrics in one column allows us to just perform just a single operation to clean the NaN values.
3. By using the pandas method 'to_numeric', the value column is casted to numeric, and the non-numeric values will be converted to NaN values, by setting errors = coerce.
4. Dropping the NaN values.
5. Converting to wide format again, by pivotting the metric and value columns.
6. The data inspections shows that all the metric columns are now float.

In [89]:
# Dividing the columns into characteristics (id_columns) and metric columns.
id_columns = ['longName', 'symbol', 'industry', 'sector']
metric_columns = ['ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow',
                  'revenueGrowth', 'operatingMargins', 'ebitda', 'debtToEquity', 'beta',
                  'returnOnEquity', 'totalCash', 'totalDebt', 'totalRevenue', 'totalCashPerShare']

# Converting to long format, by melting the metric columns.

info_df_long = info_df.melt(id_vars=id_columns, value_vars=metric_columns,
                  var_name='metric', value_name='value')

info_df_long.head(5)

,longName,symbol,industry,sector,metric,value
0,Coloplast A/S,COLO-B.CO,Medical Instruments & Supplies,Healthcare,ebitdaMargins,0.33411
1,Chr. Hansen Holding A/S,CHR.CO,Specialty Chemicals,Basic Materials,ebitdaMargins,0.33268
2,Tryg A/S,TRYG.CO,Insurance—Diversified,Financial Services,ebitdaMargins,0.12271
3,GN Store Nord A/S,GN.CO,Medical Devices,Healthcare,ebitdaMargins,0.1132
4,Rockwool A/S,ROCK-B.CO,Building Products & Equipment,Industrials,ebitdaMargins,0.16157


In [90]:
# Converting the value column to numeric, and replacing non-numbers with NaN values.

info_df_long['value'] = pd.to_numeric(info_df_long['value'], errors='coerce')

# Removing NaN values.
info_df_long.dropna()

,longName,symbol,industry,sector,metric,value
0,Coloplast A/S,COLO-B.CO,Medical Instruments & Supplies,Healthcare,ebitdaMargins,0.33411
1,Chr. Hansen Holding A/S,CHR.CO,Specialty Chemicals,Basic Materials,ebitdaMargins,0.33268
2,Tryg A/S,TRYG.CO,Insurance—Diversified,Financial Services,ebitdaMargins,0.12271
3,GN Store Nord A/S,GN.CO,Medical Devices,Healthcare,ebitdaMargins,0.11320
4,Rockwool A/S,ROCK-B.CO,Building Products & Equipment,Industrials,ebitdaMargins,0.16157
...,...,...,...,...,...,...
345,Pandora A/S,PNDORA.CO,Luxury Goods,Consumer Cyclical,totalCashPerShare,2.76300
346,Ambu A/S,AMBU-B.CO,Medical Devices,Healthcare,totalCashPerShare,0.73600
347,Danske Bank A/S,DANSKE.CO,Banks—Regional,Financial Services,totalCashPerShare,1082.19300
348,Vestas Wind Systems A/S,VWS.CO,Specialty Industrial Machinery,Industrials,totalCashPerShare,1.12900


In [91]:
# Converting to long format

info_df_wide = info_df_long.pivot(index=id_columns, columns='metric', values='value').reset_index()


In [92]:
info_df_wide.dtypes

metric
longName              object
symbol                object
industry              object
sector                object
beta                 float64
debtToEquity         float64
ebitda               float64
ebitdaMargins        float64
grossMargins         float64
operatingCashflow    float64
operatingMargins     float64
profitMargins        float64
returnOnEquity       float64
revenueGrowth        float64
totalCash            float64
totalCashPerShare    float64
totalDebt            float64
totalRevenue         float64
dtype: object

## Relational modelling.
The goal of this section is to divide the info_df into two seperate dataframes: one containing characteristics of the stocks, and another one with the financial metrics. In addition to this a company_id will be created, that will be used as the primary key for the column names, and as reference columns between the three dateframes.

1. First a df_company will be created, which will be used to create the column: company_id based on the omx_tickers list. This columns serves as an unique identifier and primary key for the table.
2. This dataframe will be merged with the info_df to add the company_id column.
3. The dataframe will then be divided into two: 
    - df_company, having only the company characteristics with company_id as primary key
    - df_financial_metrics, being a reference table having only the financial metrics and the company_id as foreign key.   
4. Lastly the df_daily dataframe will have the ticker symbol replaced with the company_id as a foreign key.  

In [93]:
#Creating a company entity based on the list. 
df_company = pd.DataFrame({"company_id": range(1, len(omx_tickers) + 1), "ticker": omx_tickers})
df_company

,company_id,ticker
0,1,COLO-B.CO
1,2,CHR.CO
2,3,TRYG.CO
3,4,GN.CO
4,5,ROCK-B.CO
5,6,GMAB.CO
6,7,FLS.CO
7,8,NOVO-B.CO
8,9,MAERSK-B.CO
9,10,DSV.CO


In [94]:
df_company_merged = pd.merge(df_company, info_df_wide, left_on='ticker', right_on='symbol')
df_company_merged.head(5)


,company_id,ticker,longName,symbol,industry,sector,beta,debtToEquity,ebitda,ebitdaMargins,grossMargins,operatingCashflow,operatingMargins,profitMargins,returnOnEquity,revenueGrowth,totalCash,totalCashPerShare,totalDebt,totalRevenue
0,1,COLO-B.CO,Coloplast A/S,COLO-B.CO,Medical Instruments & Supplies,Healthcare,0.239717,225.615,7.544000e+09,0.33411,0.68776,5.099000e+09,0.30644,0.20842,0.57181,0.188,6.330000e+08,2.982,1.870800e+10,2.257900e+10
1,2,CHR.CO,Chr. Hansen Holding A/S,CHR.CO,Specialty Chemicals,Basic Materials,0.209557,52.821,4.052000e+08,0.33268,0.55542,3.088000e+08,0.26749,0.18481,0.13049,0.118,8.280000e+07,0.629,9.634000e+08,1.218000e+09
2,3,TRYG.CO,Tryg A/S,TRYG.CO,Insurance—Diversified,Financial Services,0.341167,15.165,3.615000e+09,0.12271,0.17013,5.902000e+09,0.10282,0.08608,0.05498,0.168,1.667000e+09,2.602,6.683000e+09,2.946000e+10
3,4,GN.CO,GN Store Nord A/S,GN.CO,Medical Devices,Healthcare,1.026635,223.702,1.983000e+09,0.11320,0.50414,3.180000e+08,0.07775,0.03825,0.11108,0.241,8.700000e+08,6.799,1.611100e+10,1.751700e+10
4,5,ROCK-B.CO,Rockwool A/S,ROCK-B.CO,Building Products & Equipment,Industrials,1.065804,NaN,6.130000e+08,0.16157,0.57433,3.370000e+08,0.10438,0.06378,0.09871,0.267,2.100000e+08,9.800,0.000000e+00,3.794000e+09


In [95]:
# Creating the company table

df_company = df_company_merged[['company_id','ticker','longName','industry','sector']]

df_company.head(5)


,company_id,ticker,longName,industry,sector
0,1,COLO-B.CO,Coloplast A/S,Medical Instruments & Supplies,Healthcare
1,2,CHR.CO,Chr. Hansen Holding A/S,Specialty Chemicals,Basic Materials
2,3,TRYG.CO,Tryg A/S,Insurance—Diversified,Financial Services
3,4,GN.CO,GN Store Nord A/S,Medical Devices,Healthcare
4,5,ROCK-B.CO,Rockwool A/S,Building Products & Equipment,Industrials


In [96]:
# Creating the financial metrics dataframe 

df_financial_metrics = df_company_merged[['company_id']+ metric_columns]

# Renaming the index column to use as a primary key. 
df_financial_metrics.index.name = "financials_id"

df_financial_metrics.head(5)

,company_id,ebitdaMargins,profitMargins,grossMargins,operatingCashflow,revenueGrowth,operatingMargins,ebitda,debtToEquity,beta,returnOnEquity,totalCash,totalDebt,totalRevenue,totalCashPerShare
financials_id,,,,,,,,,,,,,,,
0,1,0.33411,0.20842,0.68776,5.099000e+09,0.188,0.30644,7.544000e+09,225.615,0.239717,0.57181,6.330000e+08,1.870800e+10,2.257900e+10,2.982
1,2,0.33268,0.18481,0.55542,3.088000e+08,0.118,0.26749,4.052000e+08,52.821,0.209557,0.13049,8.280000e+07,9.634000e+08,1.218000e+09,0.629
2,3,0.12271,0.08608,0.17013,5.902000e+09,0.168,0.10282,3.615000e+09,15.165,0.341167,0.05498,1.667000e+09,6.683000e+09,2.946000e+10,2.602
3,4,0.11320,0.03825,0.50414,3.180000e+08,0.241,0.07775,1.983000e+09,223.702,1.026635,0.11108,8.700000e+08,1.611100e+10,1.751700e+10,6.799
4,5,0.16157,0.06378,0.57433,3.370000e+08,0.267,0.10438,6.130000e+08,NaN,1.065804,0.09871,2.100000e+08,0.000000e+00,3.794000e+09,9.800


### Replacing ticker with company_id for the daily stock prices

In [97]:
#Merging the daily stock prices with df_company

df_daily_final = pd.merge(df_daily_final,df_company, left_on='Ticker', right_on='ticker', how='left')

#Selecting all the columns, with company_id instead of ticker
df_daily_final = df_daily_final[['Date','company_id','Close','Volume','DMA30','DMA50','DMA200']]
df_daily_final.head(5)


,Date,company_id,Close,Volume,DMA30,DMA50,DMA200
0,2010-01-05,22,4.961657,910560.0,NaN,NaN,NaN
1,2010-01-06,22,5.191363,51220.0,NaN,NaN,NaN
2,2010-01-07,22,5.053540,39060.0,NaN,NaN,NaN
3,2010-01-08,22,5.099480,69160.0,NaN,NaN,NaN
4,2010-01-11,22,5.191363,54420.0,NaN,NaN,NaN


## Creating a date dimension

The date table is created using code retrived from stackoverflow:
Source: https://stackoverflow.com/questions/47150709/how-to-create-a-calendar-table-date-dimension-in-pandas.

The purpose of the table is to be used as a date entity in the data model in Tableau, and is only neccessary to run once.
The code has been included in the script for demonstration purposes.

In [98]:
#Calendar table

def create_date_table(start='2010-01-01', end='2025-12-31'):
    start_ts = pd.to_datetime(start).date()
    end_ts = pd.to_datetime(end).date()

    # record timetsamp is empty for now
    dates =  pd.DataFrame(index=pd.date_range(start_ts, end_ts))
    dates.index.name = 'Date'

    days_names = {
        i: name
        for i, name
        in enumerate(['Monday', 'Tuesday', 'Wednesday',
                      'Thursday', 'Friday', 'Saturday', 
                      'Sunday'])
    }
    dates['Day_of_year'] = dates.index.dayofyear
    dates['Day'] = dates.index.dayofweek.map(days_names.get)
    dates['Week'] = dates.index.week
    dates['Month'] = dates.index.month
    dates['Quarter'] = dates.index.quarter
    dates['Year_half'] = dates.index.month.map(lambda mth: 1 if mth <7 else 2)
    dates['Year'] = dates.index.year
    dates.reset_index(inplace=True)
    dates.index.name = 'date_id'
    return dates

df_calender = create_date_table(start='2010-01-01', end='2025-12-31')
df_calender
    

/var/folders/vl/zjh9b6rs2hb_pv6wtvvf6gbw0000gn/T/ipykernel_43391/120206059.py:20: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  dates['Week'] = dates.index.week


,Date,Day_of_year,Day,Week,Month,Quarter,Year_half,Year
date_id,,,,,,,,
0,2010-01-01,1,Friday,53,1,1,1,2010
1,2010-01-02,2,Saturday,53,1,1,1,2010
2,2010-01-03,3,Sunday,53,1,1,1,2010
3,2010-01-04,4,Monday,1,1,1,1,2010
4,2010-01-05,5,Tuesday,1,1,1,1,2010
...,...,...,...,...,...,...,...,...
5839,2025-12-27,361,Saturday,52,12,4,2,2025
5840,2025-12-28,362,Sunday,52,12,4,2,2025
5841,2025-12-29,363,Monday,1,12,4,2,2025


## LOAD
Saving the four dataframes as seperate CSV

In [99]:
df_daily_final.to_csv(f"OMX_daily.csv", encoding="utf-8")

df_financial_metrics.to_csv(f'OMX_financials.csv')

df_company.to_csv(f'OMX_info.csv')
df_calender.to_csv(f'Calender.csv')

hej
